## 丢弃法的概念

在现代神经网络中，我们所指的丢弃法，通常是对输入层或者隐含层做以下操作：

* 随机选择一部分该层的输出作为丢弃元素；
* 把丢弃元素乘以0；
* 把非丢弃元素拉伸。

## 丢弃法的实现

In [1]:
import mxnet as mx
from mxnet import nd
from mxnet import gluon
from mxnet import autograd
from utils import load_data_fashion_mnist, accuracy, evaluate_accuracy, SGD

# 设置随机种子
mx.random.seed(2)

# 实现dropout
def dropout(X, drop_probability):
    # 计算保留数据的比例
    keep_probability = 1 - drop_probability
    # 确保drop_probability的输入合法
    assert 0 <= keep_probability <= 1
    # 丢弃所有元素
    if keep_probability == 0:
        return X.zeros_like()
    # 随机生成一个相同纬度的矩阵, 根据随机值和keep_probability的对比确定是否丢弃该元素
    mask = nd.random.uniform(0, 1.0, X.shape, ctx=X.context) < keep_probability
    # 保证 E[dropout(X)] == X, 对剩下的数据进行缩放
    scale = 1 / keep_probability
    return mask * X * scale

In [2]:
# 测试dropout
A = nd.arange(20).reshape((5,4))
dropout(A, 0.0)


[[  0.   1.   2.   3.]
 [  4.   5.   6.   7.]
 [  8.   9.  10.  11.]
 [ 12.  13.  14.  15.]
 [ 16.  17.  18.  19.]]
<NDArray 5x4 @cpu(0)>

In [3]:
dropout(A, 1.0)


[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
<NDArray 5x4 @cpu(0)>

In [4]:
dropout(A, 0.5)


[[  0.   2.   4.   0.]
 [  8.   0.  12.   0.]
 [ 16.  18.   0.   0.]
 [  0.   0.   0.  30.]
 [  0.  34.  36.   0.]]
<NDArray 5x4 @cpu(0)>

## 丢弃法的本质

一般来说，在集成学习里，我们可以对训练数据集有放回地采样若干次并分别训练若干个不同的分类器；测试时，把这些分类器的结果集成一下作为最终分类结果。事实上，丢弃法在模拟集成学习。丢弃法实质上是对每一个这样的数据集分别训练一个原神经网络子集的分类器。与一般的集成学习不同，这里每个原神经网络子集的分类器用的是同一套参数。因此丢弃法只是在模拟集成学习。使用丢弃法的神经网络实质上是对输入层和隐含层的参数做了正则化：学到的参数使得原神经网络不同子集在训练数据上都尽可能表现良好。

## 数据获取

In [5]:
# 批数据大小
batch_size = 256

# 加载数据
train_data, test_data = load_data_fashion_mnist(batch_size)

## 含两个隐藏层的多层感知机

In [6]:
# 模型输入大小
num_inputs = 28 * 28

# 模型输出大小
num_outputs = 10

# 第一个隐藏层节点数量
num_hidden1 = 256

# 第二个隐藏层节点数量
num_hidden2 = 256

# 随机数据时的标准差
weight_scale = 0.01

# 第一个隐藏层权重
W1 = nd.random_normal(shape=(num_inputs, num_hidden1), scale=weight_scale)
# 第一个隐藏层偏置
b1 = nd.zeros(num_hidden1)

# 第二个隐藏层权重
W2 = nd.random_normal(shape=(num_hidden1, num_hidden2), scale=weight_scale)
# 第二个隐藏层偏置
b2 = nd.zeros(num_hidden2)

# 输出层权重
W3 = nd.random_normal(shape=(num_hidden2, num_outputs), scale=weight_scale)
# 输出层偏置
b3 = nd.zeros(num_outputs)

# 参数数组
params = [W1, b1, W2, b2, W3, b3]

# 需要计算梯度, 添加自动求导
for param in params:
    param.attach_grad()

## 定义包含丢弃层的模型

In [7]:
# 第一个隐藏层的丢弃概率
drop_prob1 = 0.2
# 第二个隐藏层的丢弃概率
drop_prob2 = 0.5

# 定义网络
def net(X):
    X = X.reshape((-1, num_inputs))
    # 第一层全连接
    h1 = nd.relu(nd.dot(X, W1) + b1)
    # 在第一层全连接后添加丢弃层
    h1 = dropout(h1, drop_prob1)
    # 第二层全连接
    h2 = nd.relu(nd.dot(h1, W2) + b2)
    # 在第二层全连接后添加丢弃层
    h2 = dropout(h2, drop_prob2)
    # 返回输出
    return nd.dot(h2, W3) + b3

## 训练

In [8]:
# 定义交叉熵损失
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

# 定义学习率
learning_rate = 0.5

# 训练
for epoch in range(5):
    # 训练损失
    train_loss = 0.0
    # 训练准确率
    train_acc = 0.0
    # 迭代训练
    for data, label in train_data:
        with autograd.record():
            # 计算输出
            output = net(data)
            # 计算损失
            loss = softmax_cross_entropy(output, label)
        # 梯度反向传播
        loss.backward()
        # SGD更新梯度
        SGD(params, learning_rate / batch_size)
        # 记录训练损失
        train_loss += nd.mean(loss).asscalar()
        # 记录训练准确率
        train_acc += accuracy(output, label)
    # 计算测试准确率
    test_acc = evaluate_accuracy(test_data, net)
    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (epoch, train_loss / len(train_data), train_acc / len(train_data), test_acc))

Epoch 0. Loss: 1.221062, Train acc 0.528746, Test acc 0.754006
Epoch 1. Loss: 0.598503, Train acc 0.774890, Test acc 0.813101
Epoch 2. Loss: 0.499490, Train acc 0.818493, Test acc 0.840244
Epoch 3. Loss: 0.457343, Train acc 0.832699, Test acc 0.835036
Epoch 4. Loss: 0.426575, Train acc 0.846070, Test acc 0.849159
